In [1]:
# Import Libraries
import numpy as np

from project_utils.bb_model         import BB_Model
from project_utils.feature_stats    import Feature_Statistics_R
from project_utils.feature_stats    import Regression_Container
from project_utils.acq_data_capture import Acq_Data_nD
from project_utils.surroagte_gp     import Surrogate_GP



In [2]:
Num_Points    = 0
Title         = 'Synthetic Regression'
FUR_Weight    = 0.9
N_consistancy = 25
consist_bound = 0.3

In [3]:
if isinstance(FUR_Weight, float):
    FUR_Weight = [FUR_Weight, 1.0 - FUR_Weight]
    
print(FUR_Weight)

[0.9, 0.09999999999999998]


In [4]:
Title = Title + ' W:'+ str(FUR_Weight)

In [5]:
# Load Forrester model

Reg_Data = BB_Model(dataset = 'Synthetic_Reg', Feature_Counts=[16,12])

#Reg_Data.df().head()

In [6]:
Reg_Data.MPL()

C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.25655409810499386


MLPRegressor(random_state=RandomState(MT19937) at 0x1FBFE419840)

In [7]:
Reg_Data.Random_Forest()

0.2565731176456607


RandomForestRegressor(random_state=RandomState(MT19937) at 0x1FBFE419840)

In [8]:
Reg_Data.GP()

-1.3343059428885873


GaussianProcessRegressor()

In [9]:
Reg_Data.L_Regression()

0.17445617890500864


LinearRegression()

In [10]:
print(Reg_Data.get_features())

['Passive_0', 'Passive_1', 'Active_2', 'Active_3', 'Active_4', 'Active_5', 'Active_6', 'Active_7']


In [11]:
#Default settings
from unravel_2.ur_model import UR_Model

X_train, X_test, y_train, y_test = Reg_Data.get_TT_data()

#Initializing the explainer object
URM_Exp = UR_Model(bbox_model           = Reg_Data.get_Random_Forest(),
                   train_data           = X_train,
                   feature_names        = Reg_Data.get_features(),
                   categorical_features = Reg_Data.get_catagorical(),
                   mode                 = Reg_Data.get_mode(),
                   sampling_optimize    = "opt")

#sur_GP = Surrogate_GP()


In [12]:
#I2nd explainer object
URM_Exp_2 = UR_Model(bbox_model           = Reg_Data.get_L_Regression(),
                     train_data           = X_train,
                     feature_names        = Reg_Data.get_features(),
                     categorical_features = Reg_Data.get_catagorical(),
                     mode                 = Reg_Data.get_mode(),
                     sampling_optimize    = "opt")



In [13]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
print('Mean 0:', mean)
print('Std 0: ', std)


Mean 0: [ 15.28384412  10.19545775   8.57113813  16.29888538  12.96486435
  18.49495634 -18.75273161  16.0561291 ]
Std 0:  [1.77466075 4.86091164 1.68185932 5.46162535 3.26165975 3.80005027
 4.3571682  1.62869787]


In [ ]:
# Generate explantion data for Random Forest

Reg_Stats   = Feature_Statistics_R(feature_names = Reg_Data.get_features(),
                                   mode          = Reg_Data.get_mode(),
                                   X_train_std   = Reg_Data.get_X_train_std(),
                                   N_consistancy = N_consistancy)

Reg_Stats_2 = Feature_Statistics_R(feature_names = Reg_Data.get_features(),
                                   mode          = Reg_Data.get_mode(),
                                   X_train_std   = Reg_Data.get_X_train_std())


if Num_Points > X_test.shape[0]: Num_Points = X_test.shape[0]

elif Num_Points == 0:            Num_Points = X_test.shape[0]

for test_instance in range(Num_Points):

    print('INSTANCE: ', test_instance)
       
    data_row  = np.array([X_test[test_instance]])
    
#    data_row  = np.array([0.4])

#    print('data row ',data_row)
    
    URM_Exp.explain(X_init      = data_row.reshape(1, -1),
                    Dimension   = 'Multi D',
                    kernel_type = "Matern",
                    max_iter    = 20,
                    alpha       = "FUR_W",
                    #jitter=5,
                    interval    = 1,
                    weight      = FUR_Weight)
       
    f_predict = Reg_Data.get_Random_Forest().predict(data_row.reshape(1, -1))

    e_predict = URM_Exp.exp_predict(data_row.reshape(1, -1))
    e_predict = np.array(e_predict).ravel()
    
    acq_data = URM_Exp.get_acq_data()
    
#    X, y = URM_Exp.get_surrogate_data()

#    e_predict = sur_GP.predict(X, y)
#    e_predict = sur_GP.predict(URM_Exp.get_surrogate_data())

    print(f_predict, e_predict, acq_data.get_fe_x0())
    
    scores = URM_Exp.permutation_importance()
    
    consistancy = URM_Exp.Y_Consistancy(N_points = N_consistancy, std_bound = consist_bound)
    
    URM_Exp.del_1_rel()
    
    del_1_var = URM_Exp.get_del_1_variance()
    
    Reg_Stats.Add_Sample(sample_scores = scores,
                         outcome       = y_test[test_instance],
                         X_row         = data_row.reshape(1, -1),
                         f_prediction  = f_predict,
                         e_prediction  = e_predict.ravel(),
                         feopt         = acq_data.get_fe_x0(),
                         consistancy   = consistancy,
                         del_1_var     = del_1_var)
    
    
    ############################################################################################
    
    URM_Exp_2.explain(X_init      = data_row.reshape(1, -1),
                      Dimension   = 'Multi D',
                      kernel_type = "Matern",
                      max_iter    = 20,
                      alpha       = "FUR_W",
                      #jitter=5,
                      interval    = 1,
                      weight      = FUR_Weight)
       
    f_predict = Reg_Data.get_L_Regression().predict(data_row.reshape(1, -1))

    e_predict = URM_Exp_2.exp_predict(data_row.reshape(1, -1))
    e_predict = np.array(e_predict).ravel()
    
    scores = URM_Exp_2.permutation_importance()
    
    Reg_Stats_2.Add_Sample(sample_scores = scores,
                           outcome       = y_test[test_instance],
                           X_row         = data_row.reshape(1, -1),
                           f_prediction  = f_predict,
                           e_prediction  = e_predict.ravel(),
                           feopt         = None)
    


INSTANCE:  0
Acq_Data_nD
[-22.10739598] [-2.21073960e+01  1.00000004e-05] [-22.10739596]
Acq_Data_nD
INSTANCE:  1
Acq_Data_nD


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


[-198.05014405] [-1.98050144e+02  1.00000004e-05] [-198.05014386]
Acq_Data_nD


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


INSTANCE:  2
Acq_Data_nD


...

INSTANCE:  42
Acq_Data_nD
[-48.04727248] [-4.80472725e+01  1.00000004e-05] [-48.04727248]
Acq_Data_nD


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations

INSTANCE:  43
Acq_Data_nD


C:\Users\deneb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


[-178.54637691] [-1.78546377e+02  1.00000004e-05] [-178.5463769]
Acq_Data_nD


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
print(f_predict)
print(e_predict)


In [ ]:
Reg_Stats.Feature_Counts(max_features=20, scaled=True, threshold=0.05)


In [ ]:
Reg_Stats.Frequency_Plot()


In [ ]:
Reg_Stats.Fidelity()

In [ ]:
Reg_Stats.Jaccard_Values()

In [ ]:
Reg_Stats.Reg_Fidelity_Graph()

In [ ]:
Reg_Stats.Box_Plot()


In [ ]:
Reg_Stats.View_Explanation(instance = 0)

In [ ]:
scores = URM_Exp.permutation_importance()
    
print(scores)

In [ ]:
scores_k = URM_Exp.KL_imp()

print(scores_k)

In [ ]:
scores_v = URM_Exp.Var_imp()

print(scores_v)

In [ ]:
scores_d1 = URM_Exp.del_1_rel()

print(scores_d1)

In [ ]:
scores_d2 = URM_Exp.del_2_rel()

print(scores_d2)

In [ ]:
lin_scores = URM_Exp.Lin_scores()

print(lin_scores)

In [ ]:
URM_Exp.plot_scores(Title)

In [ ]:
print(URM_Exp.get_exp_L())


In [ ]:
acq_data.Create_BB_plot()

In [ ]:
acq_data.plot_point(0)

In [ ]:
acq_data.plot_point(5)

In [ ]:
acq_data.plot_point(10)

In [ ]:
acq_data.plot_point(15)

In [ ]:
acq_data.plot_point(19)

In [ ]:
acq_data.plot_all()

In [ ]:
acq_data.plot_t1_t2(0)

In [ ]:
acq_data.plot_t1_t2(1)

In [ ]:
acq_data.plot_t1_t2(2)

In [ ]:
acq_data.plot_t1_t2(3)

In [ ]:
acq_data.plot_t1_t2(4)

In [ ]:
acq_data.plot_t1_t2(5)

In [ ]:
acq_data.plot_t1_t2(6)

In [ ]:
acq_data.plot_t1_t2(7)

In [ ]:
acq_data.plot_t1_t2(8)

In [ ]:
acq_data.plot_t1_t2(9)

In [ ]:
acq_data.plot_t1_t2(10)

In [ ]:
acq_data.plot_t1_t2(11)

In [ ]:
acq_data.plot_t1_t2(12)

In [ ]:
acq_data.plot_t1_t2(13)

In [ ]:
acq_data.plot_t1_t2(14)

In [ ]:
acq_data.plot_t1_t2(15)

In [ ]:
acq_data.plot_t1_t2(16)

In [ ]:
acq_data.plot_t1_t2(17)

In [ ]:
acq_data.plot_t1_t2(18)

In [ ]:
acq_data.plot_t1_t2(19)

In [ ]:
Reg_Stats.Fidelity()

print('Fidelity: ', Reg_Stats.fidelity)

In [ ]:
Reg_Stats.Jaccard_Values(top_k=4)

print('Jaccard: ', Reg_Stats.jaccard_similarities)

In [ ]:
Reg_Stats.add_Feature_Coeffs(Feature_Coeffs = Reg_Data.get_Feature_Coeffs())
        
print('Coeff Ratio: ', Reg_Stats.coeffs_ratio)

In [ ]:
delete_1_var = Reg_Stats.delete_one()


In [ ]:
Reg_Stats.Consistancy(std_bound = consist_bound, plot=True, title = Title)

In [ ]:
Reg_Stats.Compare_Models (model_b = Reg_Stats_2)

In [ ]:
Reg_Stats.Regression_Calibration(plot=True, title=Title)

In [ ]:
print(Reg_Stats.calibration_MAE)

In [ ]:
Reg_Stats.Regression_Calibration_2(plot=True, title=Title)

In [ ]:
Reg_Stats.delete_one()

In [ ]:
Reg_Stats.Results_Summary(Title)